# System of Linear Equations
---
**Name:** Jonh Alexis Buot <br>
**Date:** December 2023 <br>
**Course:** CS3101N <br>
**Task:** Assignment - System of Linear Equations

---

## Assignment 1

### Code for Backwards Substitution

In [136]:
import math
import numpy as np

def IsInRowEchelonForm(A):
# =============================================================================
#   Checks whether A is in Row Echelon Form. A is a 2D numpy array.
#   A matrix in row echelon form, must satisfy two things:
#   1. Any rows which are majority zeroes must be at the bottom.
#   2. In each non-zero row, the first nonzero entry (leading entry) is in a column to the left of the leading entry below it.
# =============================================================================
    first_nonzero_indices = np.argmax(A != 0, axis=1)

    # Strip zero rows that are found at the bottom of the matrix
    first_nonzero_indices = first_nonzero_indices[:len(first_nonzero_indices) - np.argmax(first_nonzero_indices[::-1] != 0)]
    diagonal_leading_ones = all(curr < next for curr, next in zip(first_nonzero_indices[:-1], first_nonzero_indices[1:]))
    
    return diagonal_leading_ones

def BackwardsSubstitute(A):
# =============================================================================
#   Performs Backward Substitution on matrix A, if matrix A is already in
#   Row Echelon Form. If the given matrix is inconsistent, it returns none.
#   If the given matrix has infinite solutions, it returns inf, else it returns
#   the solution vector.
# =============================================================================
    assert IsInRowEchelonForm(A), "Matrix A should be in row echelon form, or reduced row echelon form."

    # Strip zero rows if the variables are equal to the number of rows, this is to account for extra-rowed consistent matrices
    buffer_A = np.copy(A)
    while np.all(buffer_A[-1] == 0):
        buffer_A = np.delete(buffer_A, -1, axis=0)
    
    if np.all(buffer_A[-1, :-1] == 0) and buffer_A[-1, -1] != 0:
        return None
    elif buffer_A.shape[0] < buffer_A.shape[1] - 1:
        return math.inf

    flipped_matrix = buffer_A[::-1, ::-1]
    solution_vector, coefficient_matrix = flipped_matrix[:, 0], flipped_matrix[:, 1:]

    for i, row in enumerate(coefficient_matrix):
        # Let's say you're solving for x in 3x + 2y + 4z = b => x = (b - 2y + 4z) / 3
        # sum of other vars is the 2y + 4z part, 
        # 3 is the scaling part (row[i]), and 
        # b is the constant (solution_vector[i]) 
        sum_of_other_vars = sum(np.delete(solution_vector, i) * np.delete(row, i))
        solution_vector[i] = (solution_vector[i] - sum_of_other_vars) / row[i]

    return solution_vector[::-1]

A = np.array([[1, -1, -2, 5],
              [0,  1,  1, 3],
              [0,  0,  1, 2]])

solution_vector = BackwardsSubstitute(A)
print(solution_vector)

[10  1  2]


### Backwards Substitution Testing

Solve the following system of equations in `Python`, the solution should be $x_1 = 1$, $x_2 = 0$, $x_3 = 2$.
$$
\begin{align}
    x_1 - x_2 + x_3 &= 3 \\
    2 x_1 + x_2 + 8 x_3 &= 18 \\
    4 x_1 + 2 x_2 - 3 x_3 &= -2 \\
\end{align}
$$

In [137]:
def RowSwap(A,k,l):
# =============================================================================
#     A is a NumPy array.  RowSwap will return duplicate array with rows
#     k and l swapped.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A)
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def RowScale(A,k,scale):
# =============================================================================
#     A is a NumPy array.  RowScale will return duplicate array with the
#     entries of row k multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A)

    for j in range(n):
        B[k][j] *= scale
        
    return B

def RowAdd(A,k,l,scale):
# =============================================================================
#     A is a numpy array.  RowAdd will return duplicate array with row
#     l modifed.  The new values will be the old values of row l added to 
#     the values of row k, multiplied by scale.
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A)
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

A = np.array([[1,-1,1,3],[2,1,8,18],[4,2,-3,-2]])

## Add -2 times row 0 to row 1
A1 = RowAdd(A, 0, 1, -2)
print("A1")
print(A1, "\n")

## Add -4 times row 0 to row 2
A2 = RowAdd(A1, 0, 2, -4)
print("A2")
print(A2, "\n")

## Add -2 times row 1 to row 2
A3 = RowAdd(A2, 1, 2, -2)
print("A3")
print(A3, "\n")

## Multiply row 1 by 1/3
A4 = RowScale(A3, 1, 1.0/3)
print("A4")
print(A4, "\n")

## Multiply row 2 by 1/19
A5 = RowScale(A4, 2, 1.0/-19.)
print("A5")
print(A5, "\n")

solution = BackwardsSubstitute(A5)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

A1
[[ 1 -1  1  3]
 [ 0  3  6 12]
 [ 4  2 -3 -2]] 

A2
[[  1  -1   1   3]
 [  0   3   6  12]
 [  0   6  -7 -14]] 

A3
[[  1  -1   1   3]
 [  0   3   6  12]
 [  0   0 -19 -38]] 

A4
[[  1  -1   1   3]
 [  0   1   2   4]
 [  0   0 -19 -38]] 

A5
[[ 1 -1  1  3]
 [ 0  1  2  4]
 [ 0  0  1  2]] 

Solution: [1 0 2] 

x_1 = 1, x_2 = 0, x_3 = 2


# Assignment 2

**1. Provide reasons why each of the following equations is, or is not, a linear equation.**

- $\cos\theta x + 4y + z = \sqrt{3} \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $\cos\theta x + 4y + z = \sqrt{3}$ is not a linear equation, since it not follow the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$, where $a_1, a_2, a_3, b$ are constants. This is because the coefficient, $\cos\theta$, of the first expression is not a constant.

- $\cos x 4y + z = \sqrt{3} \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $\cos x 4y + z = \sqrt{3}$ is not a linear equation, since it does not follow the form $a_1 x_1 + a_2 x_2 = b$, where $a_1, a_2, b$ are constants. This is because the coefficient, $4\cos x$, of the first expression is not a constant, assuming that $x_1 = y$.

- $x^{-1} + 7y + z = \sin(\frac{\pi}{2}) \rightarrow \fbox{Not a linear equation.}$ <br>
**Solution:** $x^{-1} + 7y + z = \sin(\frac{\pi}{2})$ is not a linear equation, since it does not follow the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$, where $a_1, a_2, a_3, b$ are constants. This is because not all variables are not all raised to a degree of one.

- $x^1 + y^1 + z^1 = 0 \rightarrow \fbox{Linear equation.}$ <br>
**Solution** $x^1 + y^1 + z^1 = 0$ is of the form $a_1 x_1 + a_2 x_2 + a_3 x_3 = b$ so it is a linear equation

**2. Find the solutions to the linear equations.**

- $2x_1 + 3x_2 = 5$ <br>
**Solution** <br>
We need to transform the linear equation into an `augmented matrix`. Let $x_2 = s$. By substitution, we have $2x_1 + 3s = 5$. Solving for $x_1$ yields $x_1 = \frac{5 - 3s}{2}$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{bmatrix}
        1 & 0 &\bigm| & \frac{5 - 3s}{2} \\
        0 & 1 &\bigm| & s \\
    \end{bmatrix} 
\end{align}
$
We can have any arbitrary value of $s$ that will satisfy $\frac{5 - 3s}{2}$, this arbitrary value is the solution to $x_2$, while the result from the formula $\frac{5 - 3s}{2}$ is the solution to $x_1$.

- $4x_1 + 3x_2 + 2x_3 = 1$ <br>
**Solution** <br>
We need to transform the linear equation into an `augmented matrix`. Let $x_2 = s$ and $x_3 = t$. By substitution, we have $4x_1 + 3s + 2t = 1$. Solving for $x_1$ yields $x_1 = \frac{1 - 3s - 2t}{4}$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{bmatrix}
        1 & 0 & 0 &\bigm| & \frac{1 - 3s - 2t}{4} \\
        0 & 1 & 0 &\bigm| & s \\
        0 & 0 & 1 &\bigm| & t \\
    \end{bmatrix} 
\end{align}
$
We can have any arbitrary values of $s$ and $t$ that will satisfy $\frac{1 - 3s - 2t}{4}$, these arbitrary values are the solution to $x_2$ and $x_3$, respectively, while the result from the formula $\frac{1 - 3s - 2t}{4}$ is the solution to $x_1$.

- $3x - 6y = 0$ <br>
We need to transform the linear equation into an `augmented matrix`. Let $y = s$. By substitution, we have $3x - 6s = 0$. Solving for $x$ yields $x = 2s$. An `augmented matrix` can then be created of the form: <br>
$ 
\begin{align}
    \begin{bmatrix}
        1 & 0 &\bigm| & 2s \\
        0 & 1 &\bigm| & s \\
    \end{bmatrix} 
\end{align}
$
We can have any arbitrary value of $s$ that will satisfy $2s$, this arbitrary value is the solution to $y$, while the result from the formula $2s$ is the solution to $x$.

**3. Solve the following systems of linear equations**

For manual solutions, `augmented matrices` are reduced down to reduced row echelon forms, while for `python` solutions `augmented matrices` are reduced only to row echelon forms. This is to showcase the `BackwardsSubstitute` function.

a. 
$$
\begin{align}
    x - y = 0 \\
    2x + y = 3 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
    1 & -1 &\bigm| & 0 \\
    2 &  1 &\bigm| & 3 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 & -1 &\bigm| & 0 \\
        2 &  1 &\bigm| & 3 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{bmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  3 &\bigm| & 3 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  3 &\bigm| & 3 \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{3}\text{R}_{2}}
    \begin{bmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  1 &\bigm| & 1 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & -1 &\bigm| & 0 \\
        0 &  1 &\bigm| & 1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} + \text{R}_{2}}
    \begin{bmatrix}
        1 & 0 &\bigm| & 1 \\
        0 & 1 &\bigm| & 1 \\
    \end{bmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x = 1 \text{ and } y = 1}$. <br>

**Pythonic Solution**

In [138]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, -1, 0],
              [2, 1 ,3]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 - 2R1
 [[ 1 -1  0]
 [ 0  3  3]] 

Solution: [1 1] 

x = 1, y = 1


b. 
$$
\begin{align}
    x + 5y = -1 \\
    -x + y = -5 \\
    2x + 4y = 4 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
     1 &  5 &\bigm| & -1 \\
    -1 &  1 &\bigm| & -5 \\
     2 &  4 &\bigm| &  4 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        -1 &  1 &\bigm| & -5 \\
        2 &  4 &\bigm| &  4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{1}}
    \begin{bmatrix}
        1 & 5 &\bigm| & -1 \\
        0 & 6 &\bigm| & -6 \\
        2 & 4 &\bigm| &  4 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & 5 &\bigm| & -1 \\
        0 & 6 &\bigm| & -6 \\
        2 & 4 &\bigm| &  4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} - 2 \text{R}_{1}}
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 & -6 &\bigm| &  6 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 & -6 &\bigm| &  6 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  6 &\bigm| & -6 \\
        0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{6}\text{R}_{2}}
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  5 &\bigm| & -1 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} - 5\text{R}_{2}}
    \begin{bmatrix}
        1 &  0 &\bigm| &  4 \\
        0 &  1 &\bigm| & -1 \\
        0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
\end{align}
$$

The solution is consistent since the last row is essentially $0x + 0y = 0$ which is true. <br>
$\therefore$ The solution to the systems of linear equations are $\boxed{x = 4 \text{ and } y = -1}$. <br>

**Pythonic Solution**

In [139]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 5, -1],
              [-1, 1, -5],
              [2, 4, 4]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, 1)
print("R2 + R1\n", A, "\n")

A = RowAdd(A, 0, 2, -2)
print("R3 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 + R1
 [[ 1  5 -1]
 [ 0  6 -6]
 [ 2  4  4]] 

R3 - 2R1
 [[ 1  5 -1]
 [ 0  6 -6]
 [ 0 -6  6]] 

R3 + R2
 [[ 1  5 -1]
 [ 0  6 -6]
 [ 0  0  0]] 

Solution: [ 4 -1] 

x = 4, y = -1


c. 
$$
\begin{align}
    2x_1 + 3x_2 - x_3 = -1 \\
    x_1 + x_3 = 0 \\
    -x_1 + 2x_2 -2x_3 = 0 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
     2 &  3 & -1 &\bigm| & -1 \\
     1 &  0 &  1 &\bigm| &  0 \\
    -1 &  2 & -2 &\bigm| &  0 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        -1 &  2 & -2 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{bmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        2 &  3 & -1 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} - 2 \text{R}_{2}}
    \begin{bmatrix}
        0 &  3 & -3 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        0 &  3 & -3 &\bigm| & -1 \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{3}\text{R}_{1}}
    \begin{bmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  2 & -1 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} - 2\text{R}_{1}}
    \begin{bmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} \leftrightarrow \text{R}_{2}}
    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}

    \\

    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 & -1 &\bigm| & -\frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{3}}
    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} - \text{R}_{3}}
    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  -\frac{2}{3} \\
        0 &  1 &  0 &\bigm| &  \frac{1}{3} \\
        0 &  0 &  1 &\bigm| &  \frac{2}{3} \\
    \end{bmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x_1 = -\frac{2}{3} \text{, } x_2 = \frac{1}{3} \text{, and } x_3 = \frac{2}{3}}$. <br>

**Pythonic Solution**

In [140]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[2, 3, -1, -1],
              [1, 0, 1, 0],
              [-1, 2, -2, 0]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

A = RowAdd(A, 1, 0, -2)
print("R1 - 2R2\n", A, "\n")

A = RowAdd(A, 0, 2, -2 / 3)
print("R3 - (2/3)R1\n", A, "\n")

A = RowSwap(A, 0, 1)
print("R1 <-> R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

R3 + R2
 [[ 2  3 -1 -1]
 [ 1  0  1  0]
 [ 0  2 -1  0]] 

R1 - 2R2
 [[ 0  3 -3 -1]
 [ 1  0  1  0]
 [ 0  2 -1  0]] 

R3 - (2/3)R1
 [[ 0  3 -3 -1]
 [ 1  0  1  0]
 [ 0  0  1  0]] 

R1 <-> R2
 [[ 1  0  1  0]
 [ 0  3 -3 -1]
 [ 0  0  1  0]] 

Solution: [0 0 0] 

x_1 = 0, x_2 = 0, x_3 = 0


**4. Provide reasons why the given matrices are either a row echelon, a reduced row echelon, or neither both.**

a.
$$
\begin{align}
    \begin{bmatrix}
        1 & 0 & 1 \\
        0 & 0 & 3 \\
        0 & 1 & 0 \\
    \end{bmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since the leading non-zero entry of row 2 is to the right of the leading non-zero entry of the row below it, row 3.

b.
$$
\begin{align}
    \begin{bmatrix}
        7 & 0 &  1 & 0 \\
        0 & 1 & -1 & 4 \\
        0 & 0 &  0 & 0 \\
    \end{bmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are located at the bottom of the matrix, and each leading non-zero entry of each row is to the left of the leading non-zero entry of the row below it. However, it is not in reduced row echelon form, since the leading entry of the first row is not 1, and non-zero entries can be found outside of the leading entries.

c.
$$
\begin{align}
    \begin{bmatrix}
        0 & 1 & 3 & 0 \\
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are located at the bottom of the matrix, and each leading non-zero entry of each row is to the left of the leading non-zero entry of the row below it. However, it is not in reduced row echelon form even though its leading entries are all 1, since non-zero entries can be found outside of the leading entries.

d.
$$
\begin{align}
    \begin{bmatrix}
        0 & 0 & 0 \\
        0 & 0 & 0 \\
        0 & 0 & 0 \\
    \end{bmatrix}
\end{align}
$$
This is in $\fbox{reduced row echelon form}$, since it vacuously satisfies all the necessary requirements to be one. All zero rows are at the bottom of the matrix, each leading non-zero entry are to the left of the leading non-zero entry of the row below it, all leading entries are 1, and no non-zero entries can be found outside of the leading entries.

e.
$$
\begin{align}
    \begin{bmatrix}
        1 & 0 & 3 & -4 & 0 \\
        0 & 0 & 0 &  0 & 0 \\
        0 & 1 & 5 &  0 & 1 \\
    \end{bmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since a zero row can be found in between non-zero rows, which breaks the rule that all zero rows are found at the bottom of the matrix.

f.
$$
\begin{align}
    \begin{bmatrix}
        0 & 0 & 1 \\
        0 & 1 & 0 \\
        1 & 0 & 0 \\
    \end{bmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since leading non-zero entries are found to the right of the leading non-zero entry of the row below it.

g.
$$
\begin{align}
    \begin{bmatrix}
        1 & 2 & 1 \\
        1 & 0 & 0 \\
        0 & 1 & 1 \\
        0 & 0 & 1 \\
    \end{bmatrix}
\end{align}
$$
This is $\fbox{neither in row echelon or reduced row echelon form}$, since not all leading non-zero entries are to the left of the leading non-zero entry of the row below it, specifically the ones in row 1 and row 2.

h.
$$
\begin{align}
    \begin{bmatrix}
        2 & 1 & 3 & 5 \\
        0 & 0 & 1 & -1 \\
        0 & 0 & 0 & 3 \\
        0 & 0 & 0 & 0 \\
    \end{bmatrix}
\end{align}
$$
This is in $\fbox{row echelon form}$, since all zero rows are found at the bottom of the matrix, and all leading non-zero entries are to the left of the leading non-zero entry of the row below it. However, it is not in row reduced echelon form, since not all leading non-zero entries equal to 1, nor entries outside of the leading entries are equal to 0.

**5. Solve the following systems of linear equations**

For manual solutions, `augmented matrices` are reduced down to reduced row echelon forms, while for `python` solutions `augmented matrices` are reduced only to row echelon forms. This is to showcase the `BackwardsSubstitute` function.


a. 
$$
\begin{align}
    x_1 + 2x_2 - 3x_3 = 9 \\
    2x_1 - x_2 - x_3 = 0 \\
    4x_1 - x_2 + x_3 = 4 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
    1 &  2 & -3 &\bigm| & 9 \\
    2 & -1 & -1 &\bigm| & 0 \\
    4 & -1 &  1 &\bigm| & 4 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        4 & -1 &  1 &\bigm| & 4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} - 2\text{R}_{2}}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        2 & -1 & -1 &\bigm| & 0 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 & -5 &  5 &\bigm| & -18 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 & -5 &  5 &\bigm| & -18 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} + 5 \text{R}_{3}}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  0 &  20 &\bigm| & 2 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  0 &  20 &\bigm| & 2 \\
        0 &  1 &  3 &\bigm| & 4 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} \leftrightarrow \text{R}_{3}}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  20 &\bigm| & 2 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  20 &\bigm| & 2 \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{20}\text{R}_{3}}
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}

    \\
    
    \begin{bmatrix}
        1 &  2 & -3 &\bigm| & 9 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} - 2\text{R}_{2}}
    \begin{bmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  3 &\bigm| & 4 \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} - 3\text{R}_{3}}
    \begin{bmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}

    \\

    \begin{bmatrix}
        1 &  0 & -9 &\bigm| & 1 \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} - 9\text{R}_{3}}
    \begin{bmatrix}
        1 &  0 &  0 &\bigm| & \frac{19}{10} \\
        0 &  1 &  0 &\bigm| & \frac{37}{10} \\
        0 &  0 &  1 &\bigm| & \frac{1}{10} \\
    \end{bmatrix}
\end{align}
$$

$\therefore$ The solution to the systems of linear equations are $\boxed{x_1 = \frac{19}{10} \text{, } x_2 = \frac{37}{10} \text{, and } x_3 = \frac{1}{10}}$. <br>

**Pythonic Solution**

In [141]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 2, -3, 9],
              [2, -1, -1, 0],
              [4, -1, 1, 4]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 1, 2, -2)
print("R3 - 2R2\n", A, "\n")

A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

A = RowAdd(A, 2, 1, 5)
print("R2 + 5R3\n", A, "\n")

A = RowSwap(A, 1, 2)
print("R2 <-> R3\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x_1 = {solution[0]}", f"x_2 = {solution[1]}", f"x_3 = {solution[2]}", sep=", ")

R3 - 2R2
 [[ 1  2 -3  9]
 [ 2 -1 -1  0]
 [ 0  1  3  4]] 

R2 - 2R1
 [[  1   2  -3   9]
 [  0  -5   5 -18]
 [  0   1   3   4]] 

R2 + 5R3
 [[ 1  2 -3  9]
 [ 0  0 20  2]
 [ 0  1  3  4]] 

R2 <-> R3
 [[ 1  2 -3  9]
 [ 0  1  3  4]
 [ 0  0 20  2]] 

Solution: [1 4 0] 

x_1 = 1, x_2 = 4, x_3 = 0


b. 
$$
\begin{align}
    x + 2y= -1 \\
    2x + y + z = 1 \\
    -x + y - z = -1
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
     1 &  2 &  0 &\bigm| & -1 \\
     2 &  1 &  1 &\bigm| &  1 \\
    -1 &  1 & -1 &\bigm| & -1 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        -1 &  1 & -1 &\bigm| & -1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{1}}
    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        2 &  1 &  1 &\bigm| &  1 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} - 2 \text{R}_{1}}
    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  3 & -1 &\bigm| & -2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} + \text{R}_{2}}
    \begin{bmatrix}
        1 &  2 &  0 &\bigm| & -1 \\
        0 & -3 &  1 &\bigm| &  3 \\
        0 &  0 &  0 &\bigm| &  1 \\
    \end{bmatrix}
\end{align}
$$

Looking at the last row yields $0x + 0y + 0z = 0 \neq 1$ which means this system of linear equations is inconsistent.

$\therefore$ This system of linear equations has $\fbox{no solutions}$. <br>

**Pythonic Solution**

In [142]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 2, 0, -1],
              [2, 1, 1, 1],
              [-1, 1, -1, -1]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 2, 1)
print("R3 + R1\n", A, "\n")

A = RowAdd(A, 0, 1, -2)
print("R2 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R3 + R1
 [[ 1  2  0 -1]
 [ 2  1  1  1]
 [ 0  3 -1 -2]] 

R2 - 2R1
 [[ 1  2  0 -1]
 [ 0 -3  1  3]
 [ 0  3 -1 -2]] 

R3 + R2
 [[ 1  2  0 -1]
 [ 0 -3  1  3]
 [ 0  0  0  1]] 

Solution: None 



c. 
$$
\begin{align}
    x_1 - 3x_2 - 2x_3 = 0 \\
    -x_1 + 2x_2 + x_3 = 0 \\
    2x_1 + 4x_2 + 6x_3 = 0 \\
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
     1 & -3 & -2 &\bigm| & 0 \\
    -1 &  2 &  1 &\bigm| &  0 \\
     2 &  4 &  6 &\bigm| &  0 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| & 0 \\
        -1 &  2 &  1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} + \text{R}_{1}}
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        2 &  4 &  6 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} - 2 \text{R}_{1}}
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 & 10 & 10 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 & 10 & 10 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{3} + 10 \text{R}_{2}}
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 & -1 & -1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{-1 \text{R}_{2}}
    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & -3 & -2 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{1} + 3 \text{R}_{2}}
    \begin{bmatrix}
        1 &  0 &  1 &\bigm| &  0 \\
        0 &  1 &  1 &\bigm| &  0 \\
        0 &  0 &  0 &\bigm| &  0 \\
    \end{bmatrix}
\end{align}
$$

Looking at the final row echelon form, this system of equations has infinitely many solutions. By extracting the system of equations from the `augmented matrix` we have $x + z = 0$ and $y + z = 0$. $z$ is the common variable, so it is the only thing we have control of, we can then solve everything in terms of $x$ and $y$ yielding $y = -z$ and $y = -z$.

$\therefore$ This system of linear equations has $\fbox{infinitely many solutions}$. By having any arbitrary value for $z$, we can calculate the value of $x$ and $y$ by $x = -z$ and $y = -z$, respectively. <br>

**Pythonic Solution**

In [143]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, -3, -2, 0],
              [-1, 2, 1, 0],
              [2, 4, 6, 0]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, 1)
print("R2 + R1\n", A, "\n")

A = RowAdd(A, 0, 2, -2)
print("R3 - 2R1\n", A, "\n")

A = RowAdd(A, 1, 2, 10)
print("R3 + 10R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R2 + R1
 [[ 1 -3 -2  0]
 [ 0 -1 -1  0]
 [ 2  4  6  0]] 

R3 - 2R1
 [[ 1 -3 -2  0]
 [ 0 -1 -1  0]
 [ 0 10 10  0]] 

R3 + 10R2
 [[ 1 -3 -2  0]
 [ 0 -1 -1  0]
 [ 0  0  0  0]] 

Solution: inf 




d. 
$$
\begin{align}
    w + x + 2y + z = 1 \\
    w - x - y + z = 0 \\
    x + y = 1 \\
    w + x + z = 2
\end{align}
$$

**Manual Solution** <br>
First, create an `augmented matrix` with the given systems of linear equations.
$$
\begin{bmatrix}
    1 &  1 &  2 & 1 &\bigm| &  1 \\
    1 & -1 & -1 & 1 &\bigm| &  0 \\
    0 &  1 &  1 & 0 &\bigm| &  1 \\
    1 &  1 &  0 & 1 &\bigm| &  2 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        1 &  1 &  0 & 1 &\bigm| &  2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{4} - \text{R}_{1}}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        1 & -1 & -1 & 1 &\bigm| &  0 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} - \text{R}_{1}}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 & -2 & -3 & 0 &\bigm| & -1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 & -2 & -3 & 0 &\bigm| & -1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} + 2 \text{R}_{3}}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{2} \leftrightarrow \text{R}_{3}}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 & -2 & 0 &\bigm| &  1 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_{4} - 2 \text{R}_{3}}
    \begin{bmatrix}
        1 &  1 &  2 & 1 &\bigm| &  1 \\
        0 &  1 &  1 & 0 &\bigm| &  1 \\
        0 &  0 & -1 & 0 &\bigm| &  1 \\
        0 &  0 &  0 & 0 &\bigm| & -1 \\
    \end{bmatrix}
\end{align}
$$

Looking at the last row yields $0w + 0x + 0y + 0z = 0 \neq -1$ which means this system of linear equations is inconsistent.

$\therefore$ This system of linear equations has $\fbox{no solutions}$. <br>

**Pythonic Solution**

In [144]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 1, 2, 1, 1],
              [1, -1, -1, 1, 0],
              [0, 1, 1, 0, 1],
              [1, 1, 0, 1, 2]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 3, -1)
print("R4 - R1\n", A, "\n")

A = RowAdd(A, 0, 1, -1)
print("R2 - R1\n", A, "\n")

A = RowAdd(A, 2, 1, 2)
print("R2 + 2R3\n", A, "\n")

A = RowSwap(A, 1, 2)
print("R2 <-> R3\n", A, "\n")

A = RowAdd(A, 2, 3, -2)
print("R4 - 2R3\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R4 - R1
 [[ 1  1  2  1  1]
 [ 1 -1 -1  1  0]
 [ 0  1  1  0  1]
 [ 0  0 -2  0  1]] 

R2 - R1
 [[ 1  1  2  1  1]
 [ 0 -2 -3  0 -1]
 [ 0  1  1  0  1]
 [ 0  0 -2  0  1]] 

R2 + 2R3
 [[ 1  1  2  1  1]
 [ 0  0 -1  0  1]
 [ 0  1  1  0  1]
 [ 0  0 -2  0  1]] 

R2 <-> R3
 [[ 1  1  2  1  1]
 [ 0  1  1  0  1]
 [ 0  0 -1  0  1]
 [ 0  0 -2  0  1]] 

R4 - 2R3
 [[ 1  1  2  1  1]
 [ 0  1  1  0  1]
 [ 0  0 -1  0  1]
 [ 0  0  0  0 -1]] 

Solution: None 



**6. Determine if the vectors are linear combinations**

For manual solutions, `augmented matrices` are reduced down to reduced row echelon forms, while for `python` solutions `augmented matrices` are reduced only to row echelon forms. This is to showcase the `BackwardsSubstitute` function.

a.
$$
\begin{align}
    \textbf{v} = \begin{bmatrix} 1 \\ 2 \end{bmatrix}, 
    \textbf{u}_2 = \begin{bmatrix} -1 \\ 3 \end{bmatrix}, 
    \textbf{u}_3 = \begin{bmatrix} 2 \\ -6 \end{bmatrix} 
\end{align}
$$

**Manual Solution**

Transform the vectors into a linear equation of the form $\textbf{u}_2 + \textbf{u}_3 = \textbf{v}$, by substitution we have the following:
$$
\begin{align}
    \begin{bmatrix} -1 \\ 3 \end{bmatrix}x + \begin{bmatrix} 2 \\ -6 \end{bmatrix}y = \begin{bmatrix} 1 \\ 2 \end{bmatrix} 
\end{align}
$$
This can then be translated into a system of linear equations.

$$
\begin{align}
    -x + 2y = 1 \\
    3x - 6y = 2
\end{align}
$$
This system of linear equations can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    -1 &  2 &\bigm| & 1 \\
    3 & -6 &\bigm| & 2 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        -1 &  2 &\bigm| & 1 \\
         3 & -6 &\bigm| & 2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 + 3\text{R}_1}
    \begin{bmatrix}
        -1 &  2 &\bigm| & 1 \\
         0 &  0 &\bigm| & 5 \\
    \end{bmatrix}
\end{align}
$$

Looking at the last row yields $0x + 0y = 0 \neq 5$ which means this system of linear equations is inconsistent, and yields no solution.

$\therefore$ The vectors $\textbf{v}$, $\textbf{u}_2$, and $\textbf{u}_3$ are $\fbox{not linear combinations}$ of each other. <br>

**Pythonic Solution**

In [145]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[-1, 2, 1],
              [3, -6, 2]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, 3)
print("R2 + 3R1\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R2 + 3R1
 [[-1  2  1]
 [ 0  0  5]] 

Solution: None 



b.
$$
\begin{align}
    \textbf{v} = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}, 
    \textbf{u}_2 = \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}, 
    \textbf{u}_3 = \begin{bmatrix} 0 \\  1 \\ 1 \end{bmatrix} 
\end{align}
$$

**Manual Solution**

Transform the vectors into a linear equation of the form $\textbf{u}_2 + \textbf{u}_3 = \textbf{v}$, by substitution we have the following:
$$
\begin{align}
    \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}x + \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}y = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix} 
\end{align}
$$
This can then be translated into a system of linear equations.

$$
\begin{align}
    x = 1 \\ 
    x + y = 2 \\
    y = 3 \\
\end{align}
$$
This system of linear equations can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    1 &  0 &\bigm| & 1 \\
    1 &  1 &\bigm| & 2 \\
    0 &  1 &\bigm| & 3 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        1 &  1 &\bigm| & 2 \\
        0 &  1 &\bigm| & 3 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 - \text{R}_1}
    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & 1 \\
        0 &  1 &\bigm| & 3 \\
    \end{bmatrix}

    \\

    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & 1 \\
        0 &  1 &\bigm| & 3 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 - \text{R}_2}
    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & 1 \\
        0 &  0 &\bigm| & 2 \\
    \end{bmatrix}
\end{align}
$$

Looking at the last row yields $0x + 0y = 0 \neq 2$ which means this system of linear equations is inconsistent, and yields no solution.

$\therefore$ The vectors $\textbf{v}$, $\textbf{u}_2$, and $\textbf{u}_3$ are $\fbox{not linear combinations}$ of each other. <br>

**Pythonic Solution**

In [146]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 0, 1],
              [1, 1, 2],
              [0, 1, 3]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, -1)
print("R2 - R1\n", A, "\n")

A = RowAdd(A, 1, 2, -1)
print("R3 - R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

R2 - R1
 [[1 0 1]
 [0 1 1]
 [0 1 3]] 

R3 - R2
 [[1 0 1]
 [0 1 1]
 [0 0 2]] 

Solution: None 



c.
$$
\begin{align}
    \textbf{v} = \begin{bmatrix} 3 \\ 1 \\ -2 \end{bmatrix}, 
    \textbf{u}_2 = \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}, 
    \textbf{u}_3 = \begin{bmatrix} 0 \\  1 \\ 1 \end{bmatrix} 
\end{align}
$$

**Manual Solution**

Transform the vectors into a linear equation of the form $\textbf{u}_2 + \textbf{u}_3 = \textbf{v}$, by substitution we have the following:
$$
\begin{align}
    \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}x + \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}y = \begin{bmatrix} 3 \\ 1 \\ -2 \end{bmatrix} 
\end{align}
$$
This can then be translated into a system of linear equations.

$$
\begin{align}
    x = 3 \\ 
    x + y = 1 \\
    y = -2 \\
\end{align}
$$
This system of linear equations can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    1 &  0 &\bigm| & 3 \\
    1 &  1 &\bigm| & 1 \\
    0 &  1 &\bigm| & -2 \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  0 &\bigm| & 3 \\
        1 &  1 &\bigm| & 1 \\
        0 &  1 &\bigm| & -2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 - \text{R}_1}
    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & -2 \\
        0 &  1 &\bigm| & -2 \\
    \end{bmatrix}

    \\

    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & -2 \\
        0 &  1 &\bigm| & -2 \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 - \text{R}_2}
    \begin{bmatrix}
        1 &  0 &\bigm| & 1 \\
        0 &  1 &\bigm| & -2 \\
        0 &  0 &\bigm| & 0 \\
    \end{bmatrix}
\end{align}
$$

From the results of reducing the `augmented matrix` into its reduced row echelon form, it can be seen that the solution to our linear equation are $x = 1$, and $y = -2$. By substitution, we then have a linear equation for the given vectors in the form, $\textbf{u}_2 - 2\textbf{u}_3 = \textbf{v}$.

$\therefore$ The vectors $\textbf{v}$, $\textbf{u}_2$, and $\textbf{u}_3$ are $\fbox{linear combinations}$ of each other. <br>

**Pythonic Solution**

In [147]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 0, 3],
              [1, 1, 1],
              [0, 1, -2]])

# Perform row operations to reduce it to row echelon form
A = RowAdd(A, 0, 1, -1)
print("R2 - R1\n", A, "\n")

A = RowAdd(A, 1, 2, -1)
print("R3 - R2\n", A, "\n")

solution = BackwardsSubstitute(A)
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 - R1
 [[ 1  0  3]
 [ 0  1 -2]
 [ 0  1 -2]] 

R3 - R2
 [[ 1  0  3]
 [ 0  1 -2]
 [ 0  0  0]] 

Solution: [ 3 -2] 

x = 3, y = -2


**7. Show that the following vectors are a span**

This uses the `AlgebraicExpression` class to work with algebraic expressions, which is not possible with conventional `python`.

In [157]:
from copy import deepcopy

class Algebra:
    def __init__(self, value):
        self.symbols = [(None, value)]

    @staticmethod
    def isnumber(val):
        return isinstance(val, (int, float))

    def __add__(self, other):
        expr = deepcopy(self)
        
        if other == 0: 
            return expr
        
        last_operation, last_value = self.symbols[-1]
        
        if isinstance(other, Algebra):
            last_multiply_index = len(self.symbols)
            for i, (op, _) in enumerate(reversed(self.symbols)):
                if op == '*':
                    last_multiply_index -= i
                    break

            expr.symbols.insert(last_multiply_index, ('+', deepcopy(other)))
        elif last_operation in ["+", "-"] and Algebra.isnumber(other) and Algebra.isnumber(last_value):
            last_value = -last_value if last_operation == '-' else last_value
            expr.symbols[-1] = ('+', last_value + other)
        else:
            expr.symbols.append(('+', other))
            
        return expr

    def __sub__(self, other):
        expr = deepcopy(self)
        
        if other == 0: 
            return expr
        
        last_operation, last_value = self.symbols[-1]
        
        if isinstance(other, Algebra):
            last_multiply_index = len(self.symbols)
            for i, (op, _) in enumerate(reversed(self.symbols), start=1):
                if op == '*':
                    last_multiply_index -= i
                    break
            
            expr.symbols.insert(last_multiply_index, ('-', deepcopy(other)))
        elif last_operation in ["+", "-"] and Algebra.isnumber(other) and Algebra.isnumber(last_value):
            last_value = -last_value if last_operation == '-' else last_value
            expr.symbols[-1] = ('-', last_value - other)
        else:
            expr.symbols.append(('-', other))

        return expr

    def __mul__(self, other):
        expr = deepcopy(self)

        if other == 1: 
            return expr
        
        last_operation, last_value = self.symbols[-1]
        
        if isinstance(other, Algebra):
            last_multiply_index = len(self.symbols)
            for i, (op, _) in enumerate(reversed(self.symbols), start=1):
                if op in ['+', '-']:
                    last_multiply_index -= i
                    break
            
            expr.symbols.insert(last_multiply_index, ('*', deepcopy(other)))
        elif last_operation == '*' and Algebra.isnumber(other) and Algebra.isnumber(last_value):
            expr.symbols[-1] = ('*', last_value * other)
        else:
            expr.symbols.append(('*', other))
            
        return expr
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other):
        return self - other

    def __rmul__(self, other):
        return self * other

    def __str__(self):
        expr = f"{self.symbols[0][1]}"
        for prev_symbol, symbol in zip(self.symbols[:-1], self.symbols[1:]):
            prev_oper, _ = prev_symbol
            oper, val = symbol
            
            if oper == '*' and prev_oper is None:
                expr = f"{val}{expr}" if val != -1 else f"-{expr}"
                continue
            
            if oper == '*' and prev_oper != '*':
                expr = f"({expr}) * {val}"
                continue
            
            if Algebra.isnumber(val):
                oper = '+' if val >= 0 and oper == '+' or val < 0 and oper == '-' else '-'
                val = abs(val)
            
            val = str(val)
            if val.startswith('-') and val[1].isalpha():
                oper = '-' if oper == '+' else '+'
                val = val[1:]

            expr = f"{expr} {oper} {val}"
        
        return expr
    
    def __repr__(self):
        return str(self)

a.
$$
\text{span}
\begin{pmatrix}
    \begin{bmatrix} 1 \\ 1 \end{bmatrix}
    \begin{bmatrix} 1 \\ -1 \end{bmatrix}
\end{pmatrix}
$$

**Manual Solution**

Tranform the span into a linear equation that equals $\begin{bmatrix} a \\ b \end{bmatrix} \in \mathbb{R}^2$.
$$
\begin{align}
    x\begin{bmatrix} 1 \\ 1 \end{bmatrix} +
    y\begin{bmatrix} 1 \\ -1 \end{bmatrix} =
    \begin{bmatrix} a \\ b \end{bmatrix}
\end{align}
$$
This linear equation can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    1 &  1 &\bigm| & a \\
    1 &  -1 &\bigm| & b \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 &  1 &\bigm| & a \\
        1 &  -1 &\bigm| & b \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 - \text{R}_1}
    \begin{bmatrix}
        1 &  1 &\bigm| & a \\
        0 &  -2 &\bigm| & b - a \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 &  1 &\bigm| & a \\
        0 &  -2 &\bigm| & b - a \\
    \end{bmatrix}
    \xrightarrow{-\frac{1}{2}\text{R}_2}
    \begin{bmatrix}
        1 &  1 &\bigm| & a \\
        0 &  1 &\bigm| & \frac{a - b}{2} \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 &  1 &\bigm| & a \\
        0 &  1 &\bigm| & \frac{a - b}{2} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_1 - \text{R}_2}
    \begin{bmatrix}
        1 &  0 &\bigm| & \frac{a + b}{2} \\
        0 &  1 &\bigm| & \frac{a - b}{2} \\
    \end{bmatrix}
\end{align}
$$
The linear equation results to a solution of $x = \frac{a + b}{2}$, and $y = \frac{a - b}{2}$.

$\boxed{\therefore \text{The set of vectors is a span}}$. <br>

**Pythonic Solution**

In [159]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 1, Algebra('a')],
              [1, -1, Algebra('b')]])

# Perform row operations to reduce it to reduced row echelon form
A = RowAdd(A, 0, 1, -1)
print("R2 - R1\n", A, "\n")

A = RowScale(A, 1, -1 / 2)
print("(-1/2)R2\n", A, "\n")

A = RowAdd(A, 1, 0, -1)
print("R1 - R2\n", A, "\n")

solution = A[:, -1]
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", sep=", ")

R2 - R1
 [[1 1 a]
 [0 -2 b - a]] 

(-1/2)R2
 [[1 1 a]
 [-0.0 1.0 (b - a) * -0.5]] 

R1 - R2
 [[1.0 0.0 a + (b - a) * 0.5]
 [-0.0 1.0 (b - a) * -0.5]] 

Solution: [a + (b - a) * 0.5 (b - a) * -0.5] 

x = a + (b - a) * 0.5, y = (b - a) * -0.5


The resulting expressions above, simplify nicely to the following.
$$
\begin{align}
    x &= a + (b - a) * 0.5 = a + \frac{b - a}{2} = \frac{b + a}{2} \\
    y &= (b - a) * 0.5 = \frac{b - a}{2}
\end{align}
$$
This matches the results of the manual solution.

b.
$$
\text{span}
\begin{pmatrix}
    \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}
    \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}
    \begin{bmatrix} 1 \\ 0 \\ 1 \end{bmatrix}
\end{pmatrix}
$$

**Manual Solution**

Tranform the span into a linear equation that equals $\begin{bmatrix} a \\ b \\ c \end{bmatrix} \in \mathbb{R}^3$.
$$
\begin{align}
    x\begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix} +
    y\begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix} +
    z\begin{bmatrix} 1 \\ 0 \\ 1 \end{bmatrix} =
    \begin{bmatrix} a \\ b \\ c \end{bmatrix}
\end{align}
$$
This linear equation can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    1 & 1 & 0 &\bigm| & a \\
    0 & 1 & 1 &\bigm| & b \\
    1 & 0 & 1 &\bigm| & c \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        1 & 0 & 1 &\bigm| & c \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 - \text{R}_1}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 &-1 & 1 &\bigm| & c - a \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 &-1 & 1 &\bigm| & c - a \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 + \text{R}_2}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 2 &\bigm| & c + b - a \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 2 &\bigm| & c + b - a \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{2}\text{R}_3}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 - \text{R}_3}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_1 - \text{R}_2}
    \begin{bmatrix}
        1 & 0 & 0 &\bigm| & \frac{a + c - b}{2} \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
\end{align}
$$
The linear equation results to a solution of $x = \frac{a + c - b}{2}$, $y = \frac{b + a - c}{2}$, and $z = \frac{c + b - a}{2}$.

$\boxed{\therefore \text{The set of vectors is a span}}$. <br>

**Pythonic Solution**

In [160]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 1, 0, Algebra('a')],
              [0, 1, 1, Algebra('b')],
              [1, 0, 1, Algebra('c')]])

# Perform row operations to reduce it to reduced row echelon form
A = RowAdd(A, 0, 2, -1)
print("R3 - R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

A = RowScale(A, 2, 1/2)
print("(1/2)R3\n", A, "\n")

A = RowAdd(A, 2, 1, -1)
print("R2 - R3\n", A, "\n")

A = RowAdd(A, 1, 0, -1)
print("R1 - R2\n", A, "\n")

solution = A[:, -1]
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", f"z = {solution[2]}", sep=", ")

R3 - R1
 [[1 1 0 a]
 [0 1 1 b]
 [0 -1 1 c - a]] 

R3 + R2
 [[1 1 0 a]
 [0 1 1 b]
 [0 0 2 c - a + b]] 

(1/2)R3
 [[1 1 0 a]
 [0 1 1 b]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

R2 - R3
 [[1 1 0 a]
 [0.0 1.0 0.0 b + (c - a + b) * -0.5]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

R1 - R2
 [[1.0 0.0 0.0 a + (b + (c - a + b) * -0.5) * -1]
 [0.0 1.0 0.0 b + (c - a + b) * -0.5]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

Solution: [a + (b + (c - a + b) * -0.5) * -1 b + (c - a + b) * -0.5
 (c - a + b) * 0.5] 

x = a + (b + (c - a + b) * -0.5) * -1, y = b + (c - a + b) * -0.5, z = (c - a + b) * 0.5


The resulting expressions above, simplify nicely to the following.
$$
\begin{align}
    x &= a + (b + (c - a + b) * -0.5) * -1 = a + (b - \frac{c - a + b}{2}) * -1 = a - \frac{b + a - c}{2} = \frac{a + c - b}{2} \\
    y &= b + (c - a + b) * -0.5 = b - \frac{c - a + b}{2} = \frac{b + a - c}{2} \\
    y &= (c - a + b) * 0.5 = \frac{c + b - a}{2}
\end{align}
$$
This matches the results of the manual solution.

c.
$$
\text{span}
\begin{pmatrix}
    \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix}
    \begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix}
    \begin{bmatrix} 1 \\ 0 \\ 1 \end{bmatrix}
\end{pmatrix}
$$

**Manual Solution**

Tranform the span into a linear equation that equals $\begin{bmatrix} a \\ b \\ c \end{bmatrix} \in \mathbb{R}^3$.
$$
\begin{align}
    x\begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix} +
    y\begin{bmatrix} 0 \\ 1 \\ 1 \end{bmatrix} +
    z\begin{bmatrix} 1 \\ 0 \\ 1 \end{bmatrix} =
    \begin{bmatrix} a \\ b \\ c \end{bmatrix}
\end{align}
$$
This linear equation can then be transformed into an `augmented matrix`.
$$
\begin{bmatrix}
    1 & 1 & 0 &\bigm| & a \\
    0 & 1 & 1 &\bigm| & b \\
    1 & 0 & 1 &\bigm| & c \\
\end{bmatrix}
$$
Perform row operations to reduce it to reduced row echelon form.
$$
\begin{align}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        1 & 0 & 1 &\bigm| & c \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 - \text{R}_1}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 &-1 & 1 &\bigm| & c - a \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 &-1 & 1 &\bigm| & c - a \\
    \end{bmatrix}
    \xrightarrow{\text{R}_3 + \text{R}_2}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 2 &\bigm| & c + b - a \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 2 &\bigm| & c + b - a \\
    \end{bmatrix}
    \xrightarrow{\frac{1}{2}\text{R}_3}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    
    \\

    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 1 &\bigm| & b \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_2 - \text{R}_3}
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    
    \\
    
    \begin{bmatrix}
        1 & 1 & 0 &\bigm| & a \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
    \xrightarrow{\text{R}_1 - \text{R}_2}
    \begin{bmatrix}
        1 & 0 & 0 &\bigm| & \frac{a + c - b}{2} \\
        0 & 1 & 0 &\bigm| & \frac{b + a - c}{2} \\
        0 & 0 & 1 &\bigm| & \frac{c + b - a}{2} \\
    \end{bmatrix}
\end{align}
$$
The linear equation results to a solution of $x = \frac{a + c - b}{2}$, $y = \frac{b + a - c}{2}$, and $z = \frac{c + b - a}{2}$.

$\boxed{\therefore \text{The set of vectors is a span}}$. <br>

**Pythonic Solution**

In [161]:
# Create the augmented matrix of the given systems of linear equations
A = np.array([[1, 1, 0, Algebra('a')],
              [0, 1, 1, Algebra('b')],
              [1, 0, 1, Algebra('c')]])

# Perform row operations to reduce it to reduced row echelon form
A = RowAdd(A, 0, 2, -1)
print("R3 - R1\n", A, "\n")

A = RowAdd(A, 1, 2, 1)
print("R3 + R2\n", A, "\n")

A = RowScale(A, 2, 1/2)
print("(1/2)R3\n", A, "\n")

A = RowAdd(A, 2, 1, -1)
print("R2 - R3\n", A, "\n")

A = RowAdd(A, 1, 0, -1)
print("R1 - R2\n", A, "\n")

solution = A[:, -1]
print("Solution:", solution, "\n")

print(f"x = {solution[0]}", f"y = {solution[1]}", f"z = {solution[2]}", sep=", ")

R3 - R1
 [[1 1 0 a]
 [0 1 1 b]
 [0 -1 1 c - a]] 

R3 + R2
 [[1 1 0 a]
 [0 1 1 b]
 [0 0 2 c - a + b]] 

(1/2)R3
 [[1 1 0 a]
 [0 1 1 b]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

R2 - R3
 [[1 1 0 a]
 [0.0 1.0 0.0 b + (c - a + b) * -0.5]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

R1 - R2
 [[1.0 0.0 0.0 a + (b + (c - a + b) * -0.5) * -1]
 [0.0 1.0 0.0 b + (c - a + b) * -0.5]
 [0.0 0.0 1.0 (c - a + b) * 0.5]] 

Solution: [a + (b + (c - a + b) * -0.5) * -1 b + (c - a + b) * -0.5
 (c - a + b) * 0.5] 

x = a + (b + (c - a + b) * -0.5) * -1, y = b + (c - a + b) * -0.5, z = (c - a + b) * 0.5


The resulting expressions above, simplify nicely to the following.
$$
\begin{align}
    x &= a + (b + (c - a + b) * -0.5) * -1 = a + (b - \frac{c - a + b}{2}) * -1 = a - \frac{b + a - c}{2} = \frac{a + c - b}{2} \\
    y &= b + (c - a + b) * -0.5 = b - \frac{c - a + b}{2} = \frac{b + a - c}{2} \\
    y &= (c - a + b) * 0.5 = \frac{c + b - a}{2}
\end{align}
$$
This matches the results of the manual solution.

**8. Show that the following vectors are linearly independent**

a.
$$
\begin{align}
    \begin{bmatrix} 3 \\ -1 \\ 1 \\ -1 \end{bmatrix}
    \begin{bmatrix} -1 \\ 2 \\ 1 \\ -1 \end{bmatrix}
    \begin{bmatrix} 1 \\ 1 \\ 3 \\ 1 \end{bmatrix}
    \begin{bmatrix} -1 \\ -1 \\ 1 \\ 3 \end{bmatrix}
\end{align}
$$

The vectors above are $\fbox{linearly independent}$, since there are 4 columns with 4 entries each. This is because it contradicts the definition of linear dependence, which states that for any given matrix $\textbf{A}$ of size $m \textbf{ x } n$ that is composed of $n$ column vectors of size $m$. The vectors in $\textbf{A}$ are linearly dependent $\iff m > n$. However, since in our case $m = 4$, and $n = 4$, then $m \cancel{\lt} n$. $\therefore$, the vectors are linearly independent.

b.
$$
\begin{align}
    \begin{bmatrix} 0 \\ 0 \\ 0 \\ 4 \end{bmatrix}
    \begin{bmatrix} 0 \\ 0 \\ 3 \\ 3 \end{bmatrix}
    \begin{bmatrix} 0 \\ 2 \\ 2 \\ 2 \end{bmatrix}
    \begin{bmatrix} 1 \\ 1 \\ 1 \\ 1 \end{bmatrix}
\end{align}
$$

The vectors above are $\fbox{linearly independent}$, since there are 4 columns with 4 entries each. This is because it contradicts the definition of linear dependence, which states that for any given matrix $\textbf{A}$ of size $m \textbf{ x } n$ that is composed of $n$ column vectors of size $m$. The vectors in $\textbf{A}$ are linearly dependent $\iff m > n$. However, since in our case $m = 4$, and $n = 4$, then $m \cancel{\lt} n$. $\therefore$, the vectors are linearly independent.

a.
$$
\begin{align}
    \begin{bmatrix} 1 \\ -1 \\ 1 \\ 0 \end{bmatrix}
    \begin{bmatrix} -1 \\ 1 \\ 0 \\ 1 \end{bmatrix}
    \begin{bmatrix} 1 \\ 0 \\ 1 \\ -1 \end{bmatrix}
    \begin{bmatrix} 0 \\ 1 \\ -1 \\ 1 \end{bmatrix}
\end{align}
$$

The vectors above are $\fbox{linearly independent}$, since there are 4 columns with 4 entries each. This is because it contradicts the definition of linear dependence, which states that for any given matrix $\textbf{A}$ of size $m \textbf{ x } n$ that is composed of $n$ column vectors of size $m$. The vectors in $\textbf{A}$ are linearly dependent $\iff m > n$. However, since in our case $m = 4$, and $n = 4$, then $m \cancel{\lt} n$. $\therefore$, the vectors are linearly independent.